## Pre-trained model usage

### Importing an existing deep learning model for image classification

In [ ]:
# The model for the skin cancer classifier

# Importing the libraries
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.layers.core import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
# Checking if GPU is available
# Tensorflow is installed without CUDA support
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [ ]:
# Accessing Google drive from where directories shall be accessed and created
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The paths for the training and validation images
train_path = 'drive/MyDrive/base_dir/train_dir'
valid_path = 'drive/MyDrive/base_dir/val_dir'

In [ ]:
# Declaring a few useful values
num_train_samples = 9013
num_val_samples = 1002
train_batch_size = 10
val_batch_size = 10
image_size = 224

In [ ]:
# Declaring how many steps are needed in an iteration
train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
# Setting up generators
train_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size)

valid_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size)

test_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    shuffle=False)

Found 38704 images belonging to 7 classes.
Found 1002 images belonging to 7 classes.
Found 1002 images belonging to 7 classes.


In [ ]:
# Initialising a MobileNet pre-trained model from Keras and passing it to model object
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# Making sure to exclude the top part and setting the input shape of the model to 224x224 pixels, with 3 color channels.
mobile = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
# Displaying a summary of the layers in the model
mobile.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

## Transfer learning

### Modifying MobileNet model architecture to retrain it on the data

The **MobileNet pre-trained model** has millions of connections that has already been trained on a large data set. It is reliable at identifying different features hence there is **only a need to retrain a part of it** by 'freezing' its first few layers. **After adding a fully connected layer and through only training the last few layers, a 'new' model is obtained** that can effectively identify basic features and make predictions on relevant new data.

In [ ]:
# Modifying the model
# Excluding the last 5 layers of the model
x = mobile.layers[-6].output
# Adding a dropout and dense layer for predictions
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

In [ ]:
# Creating a new model with the new outputs
model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
# Seeing a summary of the new layers in the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
# Further customising the model to include a top classifier that fits the target expected
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(7, activation='sigmoid'))

# Getting the summary of the new model
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 7, 7, 7)           1541511   
                                                                 
 flatten (Flatten)           (None, 343)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                22016     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 7)                 455       
                                                                 
Total params: 1,563,982
Trainable params: 1,533,070
Non-trainable params: 30,912
_________________________________________________________________


### Training the new model

In [ ]:
# Training the model
# Defining Top2 and Top3 Accuracy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [ ]:
# Compiling the model
new_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

# Adding weights to make the model more sensitive to melanoma, which is a type of skin cancer
class_weights={
    0: 1.0,  # akiec
    1: 1.0,  # bcc
    2: 1.0,  # bkl
    3: 1.0,  # df
    4: 3.0,  # mel
    5: 1.0,  # nv
    6: 1.0,  # vasc
}

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Declaring the filepath for the saved model
filepath = "drive/MyDrive/new_model.h5"

In [ ]:
# Declaring a checkpoint to save the best version of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1,
                             save_best_only=True, mode='max')

In [ ]:
# Reducing the learning rate as the learning stagnates
reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

callbacks_list = [checkpoint, reduce_lr]

In [ ]:
# Compiling and fitting the model. Use the Adam optimizer and crossentropical loss. 
# Using the validation data argument during fitting to include your validation data.
new_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
history = new_model.fit_generator(train_batches,
                              steps_per_epoch=train_steps,
                              class_weight=class_weights,
                              validation_data=valid_batches,
                              validation_steps=val_steps,
                              epochs=30,
                              verbose=1,
                              callbacks=callbacks_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30
902/902 [==============================] - 2125s 2s/step - loss: 2.0618 - accuracy: 0.3019 - val_loss: 2.9444 - val_accuracy: 0.0549 - lr: 0.0010
Epoch 2/30
902/902 [==============================] - 2051s 2s/step - loss: 1.7461 - accuracy: 0.4156 - val_loss: 4.3090 - val_accuracy: 0.0150 - lr: 0.0010
Epoch 3/30
902/902 [==============================] - 2072s 2s/step - loss: 1.6327 - accuracy: 0.4462 - val_loss: 3.1695 - val_accuracy: 0.0459 - lr: 0.0010
Epoch 4/30
902/902 [==============================] - 2042s 2s/step - loss: 1.5372 - accuracy: 0.4859 - val_loss: 1.6764 - val_accuracy: 0.3623 - lr: 0.0010
Epoch 5/30
902/902 [==============================] - 2030s 2s/step - loss: 1.5415 - accuracy: 0.4748 - val_loss: 2.1273 - val_accuracy: 0.2435 - lr: 0.0010
Epoch 6/30
902/902 [==============================] - 2017s 2s/step - loss: 1.5141 - accuracy: 0.4927 - val_loss: 2.5657 - val_accuracy: 0.2435 - lr: 0.0010
Epoch 7/30
231/902 [======>.......................] - ETA:

In [ ]:
### Evaluating the model using the helper function
import tensorflow

def plot_history(history : tensorflow.python.keras.callbacks.History):
    """ This helper function takes the tensorflow.python.keras.callbacks.History
    that is output from your `fit` method to plot the loss and accuracy of
    the training and validation set.
    """
    fig, axs = plt.subplots(1,2, figsize=(12,6))
    axs[0].plot(history.history['accuracy'], label='training set')
    axs[0].plot(history.history['val_accuracy'], label = 'validation set')
    axs[0].set(xlabel = 'Epoch', ylabel='Accuracy', ylim=[0, 1])

    axs[1].plot(history.history['loss'], label='training set')
    axs[1].plot(history.history['val_loss'], label = 'validation set')
    axs[1].set(xlabel = 'Epoch', ylabel='Loss', ylim=[0, 10])
    
    axs[0].legend(loc='lower right')
    axs[1].legend(loc='lower right')
    
plot_history(history)